In [2]:
import os
import pandas as pd
import glob
from datetime import datetime

In [29]:
selection = 'marketinsights'
directory = "./utils/data/Scraped News/"
file_pattern = f"{selection}_data_*.csv"
files = glob.glob(os.path.join(directory, file_pattern))
print(files)
latest_file = max(files, key=os.path.getctime)
print(latest_file)
    

['./utils/data/Scraped News/marketinsights_data_2024-06-28.csv']
./utils/data/Scraped News/marketinsights_data_2024-06-28.csv


In [44]:
from utils.marketinsights import safe_literal_eval
df = pd.read_csv(latest_file)
df.head()
#df = df.drop(['Unnamed: 0'], axis = 1)
#df.to_csv(latest_file, index = False)

,title,link,ticker,source,Article content,tables,Managers,Members of the board,Shareholders,Time
0,South Korea's SK Bioscience to buy control of ...,https://www.marketscreener.com/quote/stock/SK-...,A034730,Reuters,(Adds deal details in paragraph 2; share move ...,{'Managers': {'Managers': {0: 'Tae-Won Choi (C...,"{'Managers': {0: 'Tae-Won Choi (CEO)', 1: 'Sun...",{'Members of the board': {0: 'Tae-Won Choi (CE...,{'Name': {0: 'SK Global Chemical Singapore Pte...,2024-06-28 21:57:00
1,Birkenstock prices L Catterton entity's second...,https://www.marketscreener.com/quote/stock/BIR...,BIRK,Reuters,June 26 (Reuters) - German sandal maker Birken...,{'Managers': {'Managers': {0: 'Oliver Reichert...,"{'Managers': {0: 'Oliver Reichert (CEO)', 1: '...",{'Members of the board': {0: 'James Michael Ch...,{'Name': {0: 'Catterton Management Co. LLC (81...,2024-06-28 21:03:00
2,Bryan Sheffield-backed Tamboran raises $75 mln...,https://www.marketscreener.com/quote/stock/TAM...,TBN,Reuters,"June 26 (Reuters) - Tamboran Resources, a\nnat...","{'Managers': None, 'Members of the board': {'M...",NaN,"{'Members of the board': {}, 'Title': {}, 'Age...","{'Name': {0: 'Bryan Sheffield (11.30%)', 1: 'T...",2024-06-28 20:08:00
3,Bryan Sheffield-backed Tamboran raises $75 mln...,https://www.marketscreener.com/quote/stock/TAM...,TBN,Reuters,"June 26 (Reuters) - Tamboran Resources, a\nnat...","{'Managers': None, 'Members of the board': {'M...",NaN,"{'Members of the board': {}, 'Title': {}, 'Age...","{'Name': {0: 'Bryan Sheffield (11.30%)', 1: 'T...",2024-06-28 19:50:00
4,Australia's Tamboran Resources raises $75 mill...,https://www.marketscreener.com/quote/stock/TAM...,TBN,Reuters,"SYDNEY, June 27 (Reuters) - Australia's Tambor...","{'Managers': None, 'Members of the board': {'M...",NaN,"{'Members of the board': {}, 'Title': {}, 'Age...","{'Name': {0: 'Bryan Sheffield (11.30%)', 1: 'T...",2024-06-28 19:05:00


In [64]:

news_df = df
filtered_news_df = news_df[['Time', 'title', 'Article content', 'source', 'link', 'ticker', 'Managers', 'Members of the board', 'Shareholders']]
filtered_news_df['Managers'] = filtered_news_df['Managers'].apply(lambda x: [t[1] for t in safe_literal_eval(x)['Managers'].items()] if safe_literal_eval(x) else x)
filtered_news_df['Members of the board'] = filtered_news_df['Members of the board'].apply(lambda x: [t[1] for t in safe_literal_eval(x)['Members of the board'].items()] if safe_literal_eval(x) else x)
#filtered_news_df['Shareholders'] = filtered_news_df['Shareholders'].apply(lambda x: [t[1] for t in safe_literal_eval(x)['Shareholders'].items()] if safe_literal_eval(x) else x)

/var/folders/1s/dk_9w2pn68s9kwvxnqxs9pf00000gn/T/ipykernel_64773/3552950372.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_news_df['Managers'] = filtered_news_df['Managers'].apply(lambda x: [t[1] for t in safe_literal_eval(x)['Managers'].items()] if safe_literal_eval(x) else x)
/var/folders/1s/dk_9w2pn68s9kwvxnqxs9pf00000gn/T/ipykernel_64773/3552950372.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_news_df['Members of the board'] = filtered_news_df['Members of the board'].apply(l

In [81]:
import requests
import urllib
from bs4 import BeautifulSoup

def scrape_tables(url):
    url = url.split('news')[0] + 'company/'
    """
    Scrapes tables of managers, members of the board, and shareholders from a given URL.
    Returns a dictionary with the extracted data.
    """
    def process_names(names):
        elements = [element for element in names.split(' ') if element]
        name = elements[:-1]
        role = f'({elements[-1]})'
        return ' '.join(name) + ' ' + role

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    #print(response)
    html_content = urllib.parse.unquote(response.text)
    #print(html_content)
    soup = BeautifulSoup(html_content, 'html.parser')
    print(soup)
    return soup
    # to_find = {'Managers': None, 'Members of the board': None, 'Name': None}
    # found = {}
    # tables = soup.find_all('div', class_='card-content')
    # print(tables)
    # for table in tables:
    #     try:
    #         title = table.find('tr').find('th').get_text(strip=True)
    #         found.add(title)
    #         if title in to_find.keys() and title not in found:
    #             temp_table = table.find('table')
    #             headers = [header.get_text(strip=True) for header in table.find_all('th')]
    #             rows = temp_table.find('tbody').find_all('tr')

    #             data = []
    #             for row in rows:
    #                 cols = row.find_all('td')
    #                 data.append([col.get_text().replace('\n', '').strip() for col in cols])

    #             df = pd.DataFrame(data, columns=headers)
    #             df[title] = df[title].apply(process_names)
    #             to_find[title] = df.to_dict()
    #     except:
    #         pass
    # return to_find

In [129]:
url = 'https://www.marketscreener.com/quote/stock/BOEING-4816/news/Certain-Common-Stock-of-Public-Joint-Stock-Company-Sovcombank-are-subject-to-a-Lock-Up-Agreement-End-47273120/'
check = scrape_tables(url)

<!DOCTYPE html>
<html lang="en-US" prefix="og: https://ogp.me/ns#"><head><title>Boeing: Shareholders Board Members Managers and Company Profile | US0970231058 | MarketScreener</title><link data-version="v000013" href="https://cdn.zonebourse.com/static/compiled-css/6e30305148b59fa1d42a75c5779ef318.min.css" rel="stylesheet"/><link href="https://code.jquery.com/" rel="preconnect"/><link href="https://securepubads.g.doubleclick.net/" rel="preconnect"/><link href="https://code.highcharts.com/" rel="preconnect"/><link href="https://www.googletagmanager.com/" rel="preconnect"/><link href="https://client.px-cloud.net/" rel="preconnect"/><link href="https://cdnjs.cloudflare.com/" rel="preconnect"/><link href="https://cdn.zonebourse.com/" rel="preconnect"/><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1, user-scalable=5, shrink-to-fit=no" name="viewport"/><meta content="Boeing, stock exchange, surperformance ratings, financial ratings, shareholders, managers, business s

In [138]:
to_find = {'Managers': None, 'Members of the board': None, 'Name': None}
tables = check.find_all('div', class_='card-content')
for table in tables:
    try:
        title = table.find('tr').find('th').get_text(strip=True)
        headers = [header.get_text(strip = True) for header in table.find_all('th')]
        if title in to_find.keys():
            if title != 'Name':
                temp_table = table.find('table')
                rows = temp_table.find('tbody').find_all('tr')
                print(rows)
                data = []
                for row in rows:
                    cols = row.find_all('td')
                    data.append([col.get_text().replace('\n', '').strip() for col in cols])

                df = pd.DataFrame(data, columns=headers)
            else:
                if 'Valuation' in headers:
                    temp_table = table.find('table')
                    rows = temp_table.find('tbody').find_all('tr')
                    data = []
                    for row in rows:
                        cols = row.find_all('td')
                        data.append([col.get_text().replace('\n', ' ').strip() for col in cols])

                    df = pd.DataFrame(data, columns=headers)
                    df[title] = df[title]
                    break
    except Exception as e:
        pass
        #print(e)

[<tr id="tr_tableinsiders_UkVDUTJXWFNMdmN5c09EZU9TbE95Zz091"><td class="table-child--w240"><div class="c-flex gbetween txt-m-inline"><a href="/insider/DAVID-CALHOUN-A00VJD/" title="David Calhoun CEO Boeing">
                                David Calhoun
                            </a><span class="cs-none ml-10 js-collapse" data-collapse="tr_tableinsiders_UkVDUTJXWFNMdmN5c09EZU9TbE95Zz091" data-collapse-custom="&gt;td:not(.c-none):not(:first-child)" data-collapse-group="insiders-table"><span class="mr-15 badge">CEO</span><i class="bi-chevron-right"></i></span></div></td><td aria-expanded="true" aria-label="Title">
                    Chief Executive Officer
                </td><td aria-expanded="true" aria-label="Age" class="table-child--w80">
                    66
                </td><td aria-expanded="true" aria-label="Since" class="table-child--w110">
                                            09-06-07                                    </td></tr>, <tr id="tr_tableinsiders_UkVDU